In [3]:
!pip install torch opencv-python
!pip install git+https://github.com/ultralytics/yolov5


  Cloning https://github.com/ultralytics/yolov5 to /tmp/pip-req-build-_zgz_wao
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/yolov5 /tmp/pip-req-build-_zgz_wao
  Resolved https://github.com/ultralytics/yolov5 to commit 6629839d5f51099c46a51dc2854901a4174cecc8
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import cv2
from pathlib import Path
import os

def detect_and_save_results(image_path, output_image_path, output_txt_path):
    # Load pre-trained YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

    img = cv2.imread(image_path)

    results = model(img)

    detections = results.xyxy[0].cpu().numpy()  # [x_min, y_min, x_max, y_max, confidence, class_id]

    output_image_dir = Path(output_image_path).parent
    output_txt_dir = Path(output_txt_path).parent
    output_image_dir.mkdir(parents=True, exist_ok=True)
    output_txt_dir.mkdir(parents=True, exist_ok=True)

    with open(output_txt_path, 'w') as f:
        for detection in detections:
            x_min, y_min, x_max, y_max, conf, class_id = detection
            label = model.names[int(class_id)]
            f.write(f'{label} {conf:.2f} {x_min:.0f} {y_min:.0f} {x_max:.0f} {y_max:.0f}\n')

            cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (255, 0, 0), 2)
            cv2.putText(img, f'{label} {conf:.2f}', (int(x_min), int(y_min) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    cv2.imwrite(output_image_path, img)

image_folder = '/content/drive/My Drive/all images'
output_folder = '/content/drive/My Drive/satellite_output'

os.makedirs(output_folder, exist_ok=True)

for image_filename in os.listdir(image_folder):
    if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
        image_path = os.path.join(image_folder, image_filename)
        output_image_path = os.path.join(output_folder, f"output_{image_filename}")
        output_txt_path = os.path.join(output_folder, f"output_{image_filename.split('.')[0]}.txt")

        detect_and_save_results(image_path, output_image_path, output_txt_path)

print(f"Detection complete. Results saved to {output_folder}.")


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 153.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.1s, installed 1 package: ['gitpython>=3.1.30']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-10-9 Python-3.10.12 torch-2.4.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 281MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-9 Python-3.10.12 torch-2.4.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-

Detection complete. Results saved to /content/drive/My Drive/satellite_output.
